In [27]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import poliastro.core.elements as el
import orbital as ob
import spiceypy as sp

mu = 3.986e14 #grav parameter of earth, m3/s2

### Topocentric Observations + Times (MJD) from Champaign, IL

In [2]:
#right ascension (deg)
RA = [265.3869518, 286.0157158, 306.8358871, 113.1072289, 44.4521793, 27.5410256, \
      128.5555539, 116.9894450, 102.4669647]

#declination (deg)
DEC = [38.4539569, 47.7809854, 50.4514381, 76.1961378, 71.6883545, 61.8733178, \
       21.4050426, 32.8319850, 41.0669857]

#altitude (deg)
ALT = [19.3547079, 14.2703198, 8.1356679, 40.4709053, 23.6357032, 12.4517325, \
       10.1299902, 9.4168870, 6.8773277]

#azimuth (deg)
AZ = [55.5481428, 38.2354388, 25.9457472, 341.8114490, 350.7827597, 355.0101055, \
      289.4238109, 305.2967884, 319.7464073]

#time (MJD)
time = [60055.1340277, 60055.1354166, 60055.1368055, 60055.2104166, 60055.2118055, \
        60055.2131944, 60055.2854166, 60055.2868055, 60055.2881944]

### Initial Orbit Determination (Gauss Method, Curtis Ch. 5)

In [3]:
def pos_unit_vec(ra, dec): 
    
    ### INPUTS ###
    # ra: right ascension, degrees
    # dec: declination, degrees
    
    ### OUTPUT ###
    # pos: unit vector in the direction of the position vector
    
    ra = np.radians(ra)
    dec = np.radians(dec)
    
    pos = np.array([np.cos(dec)*np.cos(ra), np.cos(dec)*np.sin(ra), np.sin(dec)])
    
    return pos

In [4]:
#telescope coords
lat = 40.1164 #deg
long = -88.2434 #deg
elev = 233 #m
Re = 6.371e6 #m

In [5]:
#local sidereal time
theta1 = .1340277*360 + long + 360
theta2 = .1354166*360 + long + 360
theta3 = .1368055*360 + long + 360

print('theta1:', theta1)
print('theta2:', theta2)
print('theta3:', theta3)

theta1: 320.006572
theta2: 320.506576
theta3: 321.00658


In [6]:
#R vectors
#convert deg to rad
lat = np.radians(lat)
theta1 = np.radians(theta1)
theta2 = np.radians(theta2)
theta3 = np.radians(theta3)

R1 = Re*np.array([np.cos(lat)*np.cos(theta1), np.cos(lat)*np.sin(theta1), np.sin(lat)])
R2 = Re*np.array([np.cos(lat)*np.cos(theta2), np.cos(lat)*np.sin(theta2), np.sin(lat)])
R3 = Re*np.array([np.cos(lat)*np.cos(theta3), np.cos(lat)*np.sin(theta3), np.sin(lat)])

print('R1:', R1)
print('R2:', R2)
print('R3:', R3)

R1: [ 3732634.52852222 -3131322.73039057  4105106.38529905]
R2: [ 3759818.21652564 -3098630.26901548  4105106.38529905]
R3: [ 3786715.57523395 -3065701.83117302  4105106.38529905]


In [7]:
#times, MJD
t1 = time[0]
t2 = time[1]
t3 = time[2]

print('t1:', t1)
print('t2:', t2)
print('t3:', t3)

t1: 60055.1340277
t2: 60055.1354166
t3: 60055.1368055


In [8]:
#calculate time intervals (converted to seconds!!!!)
tau1 = (t1-t2)*24*60*60
tau3 = (t3-t2)*24*60*60
tau = tau3-tau1

print('tau1:', tau1)
print('tau3:', tau3)
print('tau:', tau)

tau1: -120.0009603286162
tau3: 120.00095969997346
tau: 240.00192002858967


In [9]:
#calculate cross products
rho_hat1 = pos_unit_vec(RA[0], DEC[0])
rho_hat2 = pos_unit_vec(RA[1], DEC[1])
rho_hat3 = pos_unit_vec(RA[2], DEC[2])

p1 = np.cross(rho_hat2, rho_hat3)
p2 = np.cross(rho_hat1, rho_hat3)
p3 = np.cross(rho_hat1, rho_hat2)

print('p1:', p1)
print('p2:', p2)
print('p3:', p3)

p1: [-0.12062266  0.13975096  0.15207776]
p2: [-0.2849665   0.28596121  0.33006928]
p3: [-0.17641044  0.16193868  0.18539421]


In [10]:
#calculate D0
D0 = np.dot(rho_hat1, p1)

print('D0:', D0)

D0: -0.006913558651673167


In [11]:
#compute nine scalar quantities
D11  = np.dot(R1, p1)
D12  = np.dot(R1, p2)
D13  = np.dot(R1, p3)

D21  = np.dot(R2, p1)
D22  = np.dot(R2, p2)
D23  = np.dot(R2, p3)

D31  = np.dot(R3, p1)
D32  = np.dot(R3, p2)
D33  = np.dot(R3, p3)

In [12]:
#calculate A, B
A = 1/D0 * (-D12*(tau3/tau) + D22 + D32*(tau1/tau))
B = 1/(6*D0) * (D12*(tau3**2 - tau**2)*(tau3/tau) + D32*(tau**2 - tau1**2)*(tau1/tau))

print('A:', A)
print('B:', B)

A: 10781.305924893879
B: -627437570072.1677


In [13]:
#calculate E, R2^2
E = np.dot(R2, rho_hat2)
R2_sq = np.dot(R2, R2)

print('E:', E)
print('R2^2:', R2_sq)

E: 5738580.42759604
R2^2: 40589641000000.0


In [14]:
#calculate a, b, c
a = -(A**2 + 2*A*E + R2_sq)
b = -2*mu*B*(A + E)
c = -mu**2 * B**2

print('a:', a)
print('b:', b)
print('c:', c)

a: -40713496018886.49
b: 2.875791820881495e+33
c: -6.254831704992449e+52


In [15]:
#find roots using newton's method
roots = np.roots([1, 0, a, 0, 0, b, 0, 0, c])
for i in range(len(roots)):
     if np.isreal(roots[i]) :
         print(np.real(roots[i]))
x = 6031346.291224616

-7034887.162675379
3230745.5542494534


In [16]:
#calculate slant ranges
rho1 = 1/D0*((6*(D31*tau1/tau3 + D21*tau/tau3)*x**3 \
              + mu*D31*(tau**2 - tau1**2)*tau1/tau3) \
             /(6*x**3 + mu*(tau**2 - tau3**2)) - D11)
rho2 = A + mu*B/x**3
rho3 = 1/D0*((6*(D13*tau3/tau1 - D23*tau/tau1)*x**3 \
              + mu*D13*(tau**2 - tau3**2)*tau3/tau1) \
             /(6*x**3 + mu*(tau**2 - tau1**2)) - D33)

print('rho1:', rho1)
print('rho2:', rho2)
print('rho3:', rho3)

rho1: -969839.2871834073
rho2: -1129114.1476856961
rho3: -1496188.6116889226


In [17]:
#calculate r1, r2, r3
r1 = R1 + rho1*rho_hat1
r2 = R2 + rho2*rho_hat2
r3 = R3 + rho3*rho_hat3

print('r1:', r1)
print('r2:', r2)
print('r3:', r3)

r1: [ 3793717.13689254 -2374293.96904864  3501977.36724388]
r2: [ 3550484.73254036 -2369352.67813544  3268905.18557091]
r3: [ 3215565.38269245 -2303225.73818357  2951417.50742643]


In [18]:
#calculate lagrange coefficients
f1 = 1 - 0.5*(mu/x**3)*tau1**2
f3 = 1 - 0.5*(mu/x**3)*tau3**2
g1 = tau1 - (1/6)*(mu/x**3)*tau1**3
g3 = tau3 - (1/6)*(mu/x**3)*tau3**3

In [19]:
#calculate v2
v2 = 1/(f1*g3 - f3*g1)*(-f3*r1 + f1*r3)
print('v2:', v2)

v2: [-2419.49601176   297.4120558  -2304.02723048]


In [20]:
#orbital elements
p, e, inc, raan, arg_per, true_anom = el.rv2coe(3.986e5, r2/1000, v2/1000)

In [21]:
print('semi-latus rectum:', p)
print('eccentricity:', e)
print('inclination:', np.degrees(inc))
print('right ascension of ascending node:', np.degrees(raan))
print('argument of perigee:', np.degrees(arg_per))
print('true anomaly (rad):', true_anom)

semi-latus rectum: 105.56124486218752
eccentricity: 0.9816878863972484
inclination: 136.13459396219775
right ascension of ascending node: 93.46034947422164
argument of perigee: 295.6952466232787
true anomaly (rad): -3.0896870904341274
